In [ ]:
import tkinter as tk
from tkinter import *
from tkinter import filedialog
from tkinter.filedialog import askdirectory
from tkinter import messagebox

import cv2
import matplotlib as plt
import numpy as np


import gdal
from skimage import exposure
from skimage.segmentation import quickshift
import time


window = Tk()
window.title('OBIA_in_Python')
window.iconbitmap(r'ic.ico')
window.geometry("1000x1000")


'''Creating a label Widget'''
myHeading = Label(window, text='Python Application for OBIA')
'''Print on GUI such that its always in middle'''
myHeading.grid()

def myClick(var1):
    hello = "Hello "+ var1.get()
    myLabel = Label(window, text=hello)
    var1.delete(0,'end')
    myLabel.grid(column = 1, row = 1)
    
    
# Function for opening the
# file explorer window
def browseFiles():
    global filename
    filename = filedialog.askopenfilename(initialdir = "/",
                                          title = "Select a File",
                                          filetypes = (("TIFF","*.tif*"),
                                                       ("all files","*.*")))
      
    # Change label contents
    label_file_explorer.configure(text="File Opened: "+filename)

    
def showShapefile():
    #import geopandas
    #df_gpd = gpd.read_file(plot_centroid_path)
    #df_gpd.plot()
    print("Shapefile_Function")
      


label_file_explorer = Label(window, text = "File Explorer for Raster Selection",
                            width = 100, height = 4,
                            fg = "blue")

button_explore = Button(window, text = "Browse Files", command = browseFiles)





var1 = Entry(window, width=50, font =('Helvetica',30), show='*')
var1.grid(column = 0, row = 14)

myButton = Button(window, text="Enter Password", command=myClick)
myButton.grid(column = 0, row = 15)


label_file_explorer.grid(column = 0, row = 2)
  
button_explore.grid(column = 0, row = 1)


try:
    labelfilename = Label(window, text=filename) #shows as text in the window
    labelfilename.grid(column = 0, row = 6)
    print(filename)
    
except:
    print("Not now")
    

    
myButton2 = Button(window, text="Show Vector file", command=showShapefile)
myButton2.grid(column = 0, row = 18)

myButton3 = Button(window, text="Show Raster file", command=showShapefile)
myButton3.grid(column = 0, row = 16)


"""For Drop Down Menu"""
hello = "Selection will also run the segmentation "
myLabel = Label(window, text=hello)
myLabel.grid(column = 1, row = 5)

def seg(choice):
    global outputfilename
    outputfoldername = filedialog.askdirectory()
    print (outputfoldername)
    
    if choice=="kmeans_Segmentation":
        path = filename
        img = cv2.imread(path)

        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        twoDimage = img.reshape((-1,3))
        twoDimage = np.float32(twoDimage)
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
        K = 2
        attempts=10
        ret,label,center=cv2.kmeans(twoDimage,K,None,criteria,attempts,cv2.KMEANS_PP_CENTERS)
        center = np.uint8(center)
        res = center[label.flatten()]
        result_image = res.reshape((img.shape))
        cv2.imwrite(str(outputfoldername)+ '/segmented_result_Kmeans.tif',result_image)

        
    elif choice=="Chessboard":  
        myLabel = Label(window, text=choice)
        myLabel.grid(column = 1, row = 16)
        
        print(button_explore)
    
    elif choice=="QuickShift" or "QuickShift_ratio" or "slic": 
        myLabel = Label(window, text=choice)
        myLabel.grid(column = 1, row = 16)

        naip_fn = filename
        driverTiff = gdal.GetDriverByName('GTiff')
        naip_ds = gdal.Open(naip_fn)
        nbands = naip_ds.RasterCount
        band_data = []
        print('bands', naip_ds.RasterCount, 'rows', naip_ds.RasterYSize, 'columns',
              naip_ds.RasterXSize)
        for i in range(1, nbands+1):
            band = naip_ds.GetRasterBand(i).ReadAsArray()
            band_data.append(band)
        band_data = np.dstack(band_data)
        # scale image values from 0.0 - 1.0
        img = exposure.rescale_intensity(band_data)

        # do segmentation multiple options with quickshift and slic
        seg_start = time.time()
        if choice=="Quickshift":
            segments = quickshift(img, convert2lab=False)
            
        elif choice=="Quickshift_ratio":
            segments = quickshift(img, ratio=0.8, convert2lab=False)
            
        elif choice=="slic":    
            segments = slic(img, n_segments=100000, compactness=0.1)

        #segments = slic(img, n_segments=500000, compactness=0.1)
        print('segments complete', time.time() - seg_start)
        # save segments to raster
        segments_fn = str(outputfoldername)+ '/segmented_result_'+str(choice)+'.tif'
        segments_ds = driverTiff.Create(segments_fn, naip_ds.RasterXSize, naip_ds.RasterYSize, 1, gdal.GDT_Float32)
        segments_ds.SetGeoTransform(naip_ds.GetGeoTransform())
        segments_ds.SetProjection(naip_ds.GetProjectionRef())
        segments_ds.GetRasterBand(1).WriteArray(segments)
        #Flush Cache here
        segments_ds = None
        del(segments_ds)
        #segments_ds_fn.SyncToDisk()
    
    messagebox.showinfo('Information', 'Segmentation Done Check output Folder')


OptionList = ["kmeans_Segmentation","Chessboard","QuickShift", "QuickShift_ratio", "slic"]

variable = tk.StringVar(window)
variable.set(OptionList[0])

opt = tk.OptionMenu(window, variable, *OptionList, command=seg)
opt.config(width=90, font=('Helvetica', 12))
opt.grid(column = 0, row = 5)
print(opt)


"Show Canvas"
myCanvas = Canvas(window, width=500,height=500, background="White")
myCanvas.grid(column = 0, row = 19)




scroll_bar = Scrollbar(window)

scroll_bar.grid(column = 2, row=1)


window.mainloop()


E:/GUI/data/image/ESAWorldCover_imageToDriveExample_transform.tif
.!optionmenu
